In [1]:
import pandas as pd
df = pd.read_csv('NBA_Season_Stats.csv')

df = df.drop(df.index[24624:]) #indicies beyond are per game instead of per season
df.reset_index
df.tail


<bound method NDFrame.tail of        Unnamed: 0    Year             Player  Pos   Age   Tm     G    GS  \
0               0  1950.0    Curly Armstrong  G-F  31.0  FTW  63.0   NaN   
1               1  1950.0       Cliff Barker   SG  29.0  INO  49.0   NaN   
2               2  1950.0      Leo Barnhorst   SF  25.0  CHS  67.0   NaN   
3               3  1950.0         Ed Bartels    F  24.0  TOT  15.0   NaN   
4               4  1950.0         Ed Bartels    F  24.0  DNN  13.0   NaN   
...           ...     ...                ...  ...   ...  ...   ...   ...   
24619       24686  2017.0        Cody Zeller   PF  24.0  CHO  62.0  58.0   
24620       24687  2017.0       Tyler Zeller    C  27.0  BOS  51.0   5.0   
24621       24688  2017.0  Stephen Zimmerman    C  20.0  ORL  19.0   0.0   
24622       24689  2017.0        Paul Zipser   SF  22.0  CHI  44.0  18.0   
24623       24690  2017.0        Ivica Zubac    C  19.0  LAL  38.0  11.0   

           MP     FG  ...    FT%    ORB    DRB    TRB    

In [2]:
#data cleaning->need to replace missing values needed for PER calculation
#used this site to help clean: https://www.basketball-reference.com/about/per.html

df['3P'] = df['3P'].fillna(0)
df['3PA'] = df['3PA'].fillna(0)
df['3P%'] = df['3P%'].fillna(0)
df['STL'] = df['STL'].fillna(0)
df['BLK'] = df['BLK'].fillna(0)
df['TOV'] = df['TOV'].fillna(0)

df['ORB'] = df['ORB'].fillna(.3 * df['TRB'])
df['DRB'] = df['DRB'].fillna(.7 * df['TRB'])

df['PER'] = ((df['FG'] * 85.910) + (df['STL'] * 53.897) + (df['3P']* 51.757 )+ (df['FT']* 46.845 )+ (df['BLK'] * 39.190) + (df['ORB'] * 39.190) + (df['AST'] * 34.677 )+ (df['DRB'] * 14.707 )- (df['PF'] * 17.174) - ((df['FTA']-df['FT']) * 20.091) - ((df['FGA']-df['FG']) * 39.190) - (df['TOV'] * 53.897))/df['MP'] 


df['PER'] = df['PER'].fillna(15) #estimates players prior to 1952 to be avg (15)


#df.to_csv("NBA_STATS_WITH_PER.csv")

In [3]:
#Add player IDs->differentiate repeated player names
df['ID'] = df['Player']
names_dict = {}
for idx, name in enumerate(df['Player']):

    if name not in names_dict:
        names_dict[name] = 1
    else:
        names_dict[name] = names_dict[name] + 1
    

    
    value = str(df.at[idx, 'Player'])+"_"+str(names_dict[name])
    df.at[idx, 'ID'] = value

#add col to front
id_col = df.pop('ID')

df.insert(0, 'ID', id_col)

df = df.drop(df.columns[1], axis =1) #drop redundant index col
df
df.to_csv("NBA_STATS_WITH_PER.csv") #added IDs to same csv